# Enviroweather Private Weather Stations Python package: `ewx_pws`
## Example usage

Using this package in a project to pull weather data you start as follows

In [3]:

from ewx_pws import __version__, ewx_pws

print(__version__)

0.1.0


This package works with the following station types:

In [4]:
print(ewx_pws.STATION_TYPES)

['DAVIS', 'CAMPBELL', 'ONSET', 'RAINWISE', 'SPECTRUM', 'ZENTRA']


The main function accepts configuration information for one or more weather stations API (clouds), 
and uses that to connect to the API for that station and pull data for a specific time interval. 

The configuration can be saved in a CSV file, in the environment (OS variables), or in the future in a database table. 

This example usage notebook assumes the values are in the environment.  This can be acheived by putting the 
configuration in a file named `.env` in the root of this folder.    See documentation for Python dotenv package for more info

The file must be in the format :  (TBD)

The main package automatically looks for and loads this file. 

In [5]:
# load weather station configuration from an environment file
from dotenv import load_dotenv
load_dotenv()
stations = ewx_pws.stations_from_env()

Get sample weather from the previous 15 minute period (e.g. if it's 1:17, then from 1:00 to 1:15)

In [6]:
# get weather for these stations, leveraging the MultiweatherAPI python package

weather_readings = ewx_pws.get_readings(stations)


(datetime.datetime(2023, 2, 21, 15, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2023, 2, 21, 15, 15, tzinfo=datetime.timezone.utc))
UTC Start date: 2023-02-21 15:00:00+00:00
Local time Start date: 2023-02-21 10:00:00-05:00
UTC End date: 2023-02-21 15:15:00+00:00, local time zone: ET
Local time End date: 2023-02-21 10:15:00-05:00
Parameter name: "api-key" has value "aoqdcbirudd1sarq6erfj6tgtw67sbvo"
Parameter name: "end-timestamp" has value "1676992500"
Parameter name: "start-timestamp" has value "1676991600"
Parameter name: "station-id" has value "117005"
Parameter name: "t" has value "1676993084"
Data string to hash is: "api-keyaoqdcbirudd1sarq6erfj6tgtw67sbvoend-timestamp1676992500start-timestamp1676991600station-id117005t1676993084"
API Signature is: "f11171703e90585fe03e6a74da893f93111785fa418a62500c5a19c84863701d"
(1676991600, 1676992500, 'f11171703e90585fe03e6a74da893f93111785fa418a62500c5a19c84863701d')
2023-02-21 15:05:00+00:00
2023-02-21 15:10:00+00:00
2023-02-21 15:15

The output from MultiweatherAPI is a mix of the JSON output from the stations, and transformed data.  

let's look at just the transformed

In [8]:
for station_key in weather_readings:
    reading = weather_readings[station_key]
    print(f"Data from station {reading['station_id']}")
    print("----------")
    print(reading['data'])
    

Data from station DAVIS_1
----------
[{'station_id': '117005', 'request_datetime': '2023-02-21 10:24:44', 'data_datetime': '2023-02-21 15:05:00', 'atemp': -0.56, 'pcpn': 0.0, 'relh': 70.0}, {'station_id': '117005', 'request_datetime': '2023-02-21 10:24:44', 'data_datetime': '2023-02-21 15:10:00', 'atemp': -0.61, 'pcpn': 0.0, 'relh': 69.0}, {'station_id': '117005', 'request_datetime': '2023-02-21 10:24:44', 'data_datetime': '2023-02-21 15:15:00', 'atemp': -0.83, 'pcpn': 0.0, 'relh': 68.0}]
Data from station ONSET_1
----------
[{'station_id': None, 'request_datetime': None, 'data_datetime': '2023-02-21 15:00:00', 'atemp': None, 'pcpn': None, 'relh': None}, {'station_id': None, 'request_datetime': None, 'data_datetime': '2023-02-21 15:05:00', 'atemp': None, 'pcpn': None, 'relh': None}, {'station_id': None, 'request_datetime': None, 'data_datetime': '2023-02-21 15:10:00', 'atemp': None, 'pcpn': None, 'relh': None}, {'station_id': None, 'request_datetime': None, 'data_datetime': '2023-02-21